<a href="https://colab.research.google.com/github/divyasri-22/image-recognization/blob/main/SSLIMAGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%writefile app.py
import streamlit as st
from PIL import Image
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

st.set_page_config(page_title="SSL Image Recognition", page_icon="🧠", layout="wide")
st.title("🔍 Self-Supervised Learning: Image Recognition & Similarity Demo")
st.write("**Upload two images to see how similar they are based on embeddings from ResNet50.**")

@st.cache_resource
def load_model():
    model = models.resnet50(pretrained=True)
    model.eval()
    return model

model = load_model()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

col1, col2 = st.columns(2)
with col1:
    img1_file = st.file_uploader("Upload first image", type=["jpg", "jpeg", "png"], key="1")
with col2:
    img2_file = st.file_uploader("Upload second image", type=["jpg", "jpeg", "png"], key="2")

def get_embedding(image):
    img_t = transform(image).unsqueeze(0)
    with torch.no_grad():
        embedding = model(img_t).squeeze().numpy()
    return embedding

if img1_file and img2_file:
    img1 = Image.open(img1_file).convert('RGB')
    img2 = Image.open(img2_file).convert('RGB')

    col1, col2 = st.columns(2)
    with col1:
        st.image(img1, caption="Image 1", use_container_width=True)
    with col2:
        st.image(img2, caption="Image 2", use_container_width=True)

    with st.spinner("Generating embeddings..."):
        emb1 = get_embedding(img1)
        emb2 = get_embedding(img2)

    similarity = cosine_similarity([emb1], [emb2])[0][0]
    similarity_score = round(similarity * 100, 2)

    st.subheader(f"Similarity Score: **{similarity_score}%**")
    if similarity_score > 70:
        st.success("These images are quite similar!")
    elif similarity_score > 40:
        st.warning("Somewhat similar.")
    else:
        st.error("Not very similar.")

    st.write("### Embedding Visualization (Top 20 values of Image 1)")
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.bar(range(20), emb1[:20])
    ax.set_title("Embedding Values (Top 20)")
    st.pyplot(fig)


Overwriting app.py


In [4]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


In [ ]:
!pip install streamlit -q
!streamlit run app.py & sleep 5 && ./cloudflared tunnel --url http://localhost:8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.187.147.143:8501

2025-07-27T13:43:02Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-07-27T13:43:02Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-07-27T13:43:06Z INF +--------------------------------------------------------------------------------------------+
2025-07-27T13:43:06Z INF